In [1]:
# Import dependencies
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import unidecode 
from config import api_key
import requests
import json

In [2]:
breeds_url = 'https://stacker.com/stories/2454/origins-50-most-popular-dog-breeds'

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/Users/ellengrove/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


In [3]:
browser.visit(breeds_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [4]:
results = soup.find_all('div', class_='ct-slideshow__slide__text-container')
results.pop(0)

<div class="ct-slideshow__slide__text-container">
<h2 class="ct-slideshow__slide__text-container__caption">
<div>Origins of the 50 most popular dog breeds</div>
</h2>
<div class="ct-slideshow__slide__text-container__description">
<div><p>Dogs have long been known as man's best friend. Science tells us why: European scientists studying the DNA of prehistoric dogs in Germany concluded in 2017 that all modern dogs <a href="https://www.sciencedaily.com/releases/2017/07/170718113516.htm" rel="noopener" target="_blank">have the same geographic origin</a> and can be traced back to a single instance of domestication about 20,000 to 40,000 years ago. This makes them the first animal domesticated by man. The mutually beneficial relationship between human and dog became inseparable from there, as dogs provided safety and security to their nomadic hunter-gatherer owners and humans returned the favor with food and shelter. And in each other, both parties found loyal companionship.<br/><br/>
Modern 

In [5]:
len(results)

50

In [6]:
# initialize empty list to store breeds
breed_list = []

for item in results:
    # initialize empty dictionary to hold breed info
    breed_dict = {}
    dict_item = {}
    # find breed name and rank and add to dictionary
    breed_title = item.find(class_="ct-slideshow__slide__text-container__caption").text
    breed_rank = breed_title.split('. ',1)
    breed_dict['rank'] = breed_rank[0].strip()
    breed_dict['breed'] = unidecode.unidecode(breed_rank[1].strip())
    # find origin and add to dictionary
    para = item.find('div',class_="ct-slideshow__slide__text-container__description").\
        get_text(separator='<br>', strip=True).split('<br>')
    origin = para[0].split(': ',1)
    breed_dict['origin'] = origin[1]
    # add breed info to list
    try: 
        location = breed_dict['origin']
        url = f"https://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}"
        coordinates = requests.get(url).json()
    
        pair = [coordinates['coord']['lat'],coordinates['coord']['lon']]
        
    except:
        pair = ['NULL','NULL']
    
    breed_dict['origin_coords'] = pair
    
#     dict_item['breed_info'] = breed_dict
    breed_list.append(breed_dict)

In [7]:
dog_data = {}
dog_data['top_dog_breeds'] = breed_list
dog_data

{'top_dog_breeds': [{'rank': '#50',
   'breed': 'English cocker spaniel',
   'origin': 'England',
   'origin_coords': [34.5443, -91.969]},
  {'rank': '#49',
   'breed': 'Portuguese water dog',
   'origin': 'Portugal',
   'origin_coords': [39.5, -8]},
  {'rank': '#48',
   'breed': 'St. Bernard',
   'origin': 'Switzerland',
   'origin_coords': [47.0002, 8.0143]},
  {'rank': '#47',
   'breed': 'Akita',
   'origin': 'Japan',
   'origin_coords': [35.6854, 139.7531]},
  {'rank': '#46',
   'breed': 'Chesapeake Bay retriever',
   'origin': 'Maryland, USA',
   'origin_coords': [39.0004, -76.75]},
  {'rank': '#45',
   'breed': 'Shiba Inu',
   'origin': 'Japan',
   'origin_coords': [35.6854, 139.7531]},
  {'rank': '#44',
   'breed': 'West Highland white Terrier',
   'origin': 'Scotland',
   'origin_coords': [34.8335, -79.4998]},
  {'rank': '#43',
   'breed': 'Bichon Frise',
   'origin': 'Spain',
   'origin_coords': [40, -4]},
  {'rank': '#42',
   'breed': 'Rhodesian ridgeback',
   'origin': 'Afri

In [8]:
with open('data/top_dogs.json', 'w') as outfile:
    json.dump(dog_data, outfile,indent=4)